In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dropout, Dense, Conv1D, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, Reshape, Flatten

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"
!kaggle datasets download -d selfishgene/historical-hourly-weather-data

 84% 10.0M/12.0M [00:00<00:00, 16.5MB/s]
100% 12.0M/12.0M [00:00<00:00, 13.0MB/s]


Unzip it to /content.

In [ ]:
!unzip -qq /content/historical-hourly-weather-data.zip

In [ ]:
import pandas as pd
import os
from datetime import datetime

city = "Los Angeles"
window_size = 24

lowest_pressure = 850 # lowest than lowest pressure (hPa) recorded ever
highest_pressure = 1090

train_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
root_path = os.path.join('/content/drive/MyDrive/projects/weather/', train_date)
if not os.path.isdir(root_path): os.makedirs(root_path)

df_temp = pd.read_csv('temperature.csv', usecols = [city])
df_temp = df_temp - 273.15 # kelvin to celsius
df_humid = pd.read_csv('humidity.csv', usecols = [city])
df_pres = pd.read_csv('pressure.csv', usecols = [city])
df_pres = df_pres - lowest_pressure

df_list = [df_temp, df_humid, df_pres]
missing_rows = []

for df in df_list:
  print(df.isnull().values.sum())

for df in df_list:
  df.drop(df.tail(793).index, inplace=True)
  df.drop(df.head(1).index, inplace=True)
  df.reset_index(inplace = True, drop = True)
  print(df)
  print("Indexes of nan values:", list(df.loc[pd.isna(df[city]), :].index))

3
152
252
       Los Angeles
0        18.720000
1        18.718186
2        18.712844
3        18.707503
4        18.702162
...            ...
44454    25.530000
44455    26.840000
44456    28.170000
44457    28.450000
44458    28.610000

[44459 rows x 1 columns]
Indexes of nan values: [3858, 3859]
       Los Angeles
0             88.0
1             88.0
2             88.0
3             88.0
4             88.0
...            ...
44454         24.0
44455         20.0
44456         37.0
44457         18.0
44458         34.0

[44459 rows x 1 columns]
Indexes of nan values: [365, 389, 390, 391, 393, 408, 480, 534, 535, 593, 598, 607, 629, 640, 675, 841, 842, 843, 844, 988, 991, 998, 1013, 1014, 1016, 1022, 1036, 1060, 1064, 1065, 1084, 1302, 1303, 1304, 1326, 1710, 1732, 1737, 2118, 2131, 2161, 2195, 2196, 2198, 2202, 2211, 2213, 2215, 2237, 2259, 2268, 2273, 2274, 2277, 2278, 2279, 2375, 2376, 2408, 2536, 2543, 2583, 2596, 2697, 2698, 2728, 2755, 2906, 3005, 3032, 3119, 3178, 3223, 3240, 

In [ ]:
for df in df_list[:3]:
  df.interpolate(inplace=True)
  print(df)
  print(list(df.loc[pd.isna(df[city]), :].index))

       Los Angeles
0        18.720000
1        18.718186
2        18.712844
3        18.707503
4        18.702162
...            ...
44454    25.530000
44455    26.840000
44456    28.170000
44457    28.450000
44458    28.610000

[44459 rows x 1 columns]
[]
       Los Angeles
0             88.0
1             88.0
2             88.0
3             88.0
4             88.0
...            ...
44454         24.0
44455         20.0
44456         37.0
44457         18.0
44458         34.0

[44459 rows x 1 columns]
[]
       Los Angeles
0            163.0
1            163.0
2            163.0
3            163.0
4            163.0
...            ...
44454        164.0
44455        164.0
44456        163.0
44457        162.0
44458        161.0

[44459 rows x 1 columns]
[]


In [ ]:
frames = [df_temp[:-window_size], df_humid[:-window_size], df_pres[:-window_size]]
df = pd.concat(frames, axis=1)
df.columns = ['temp', 'humid', 'pres']

print("First 50 rows: ", df[0:50])
print("Last 50 rows: ", df[-50:])

print(df['temp'].isnull().sum())
print(df['humid'].isnull().sum())
print(df['pres'].isnull().sum())

First 50 rows:           temp  humid   pres
0   18.720000   88.0  163.0
1   18.718186   88.0  163.0
2   18.712844   88.0  163.0
3   18.707503   88.0  163.0
4   18.702162   88.0  163.0
5   18.696821   88.0  163.0
6   18.691480   88.0  163.0
7   18.686139   88.0  163.0
8   18.680798   88.0  163.0
9   18.675457   88.0  163.0
10  18.670116   88.0  163.0
11  18.664775   88.0  163.0
12  18.659434   88.0  163.0
13  18.654093   88.0  163.0
14  18.648752   88.0  163.0
15  18.643411   88.0  163.0
16  18.638070   88.0  163.0
17  18.632729   88.0  163.0
18  18.627387   88.0  163.0
19  18.622046   88.0  163.0
20  18.616705   88.0  163.0
21  18.611364   88.0  163.0
22  18.606023   88.0  163.0
23  18.600682   88.0  163.0
24  18.595341   88.0  163.0
25  18.590000   88.0  163.0
26  20.820000   88.0  163.0
27  22.440000   23.0  164.0
28  24.780000   19.0  164.0
29  30.680000   24.0  163.0
30  31.880000   15.0  163.0
31  32.970000   12.0  162.0
32  34.510000   14.0  162.0
33  34.560000   22.0  161.0
34  

In [ ]:
df_test = df[:100]

print(df_test[:])

         temp  humid   pres
0   18.720000   88.0  163.0
1   18.718186   88.0  163.0
2   18.712844   88.0  163.0
3   18.707503   88.0  163.0
4   18.702162   88.0  163.0
..        ...    ...    ...
95  16.670000   77.0  167.0
96  17.100000   77.0  168.0
97  17.260000   72.0  168.0
98  17.500000   72.0  169.0
99  18.640000   64.0  169.0

[100 rows x 3 columns]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

data_array = df.to_numpy().astype('float32')

data_array[:, 0] /= 60
data_array[:, 1] /= 100
data_array[:, 2] /= (highest_pressure - lowest_pressure)

X = np.zeros(shape = (data_array[:, 2].shape[0]-window_size, window_size*3))

for i in range(len(data_array)-window_size):
  X[i] = data_array[i:i+window_size, 0:3].flatten()

y_temp = data_array[:-window_size, 0]
y_humi = data_array[:-window_size, 1]

y = data_array[:-window_size, :2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

y_temp_train = y_train[:, 0]
y_humi_train = y_train[:, 1]

y_temp_test = y_test[:, 0]
y_humi_test = y_test[:, 1]

X_train = np.reshape(X_train, (-1, window_size, 3))
X_test = np.reshape(X_test, (-1, window_size, 3))

print("Train set shapes: ", X_train.shape, y_temp_train.shape, y_humi_train.shape)
print("Test set shapes: ", X_test.shape, y_temp_test.shape, y_humi_test.shape)
print("Last training window X: \n", X_train[-window_size:])
print("Last training window temperature: \n", y_temp_train[-window_size:])
print("Last training window humidity: \n", y_humi_train[-window_size:])

Train set shapes:  (39969, 24, 3) (39969,) (39969,)
Test set shapes:  (4442, 24, 3) (4442,) (4442,)
Last training window X: 
 [[[0.13033333 0.69999999 0.69999999]
  [0.13149999 0.93000001 0.70416665]
  [0.13016666 0.81       0.70416665]
  ...
  [0.07716667 0.86000001 0.6875    ]
  [0.0675     0.86000001 0.6875    ]
  [0.06366666 0.80000001 0.6875    ]]

 [[0.133      0.56999999 0.70833331]
  [0.133      0.56999999 0.70833331]
  [0.11933333 0.56999999 0.70833331]
  ...
  [0.1855     0.34999999 0.72083336]
  [0.183      0.34999999 0.72083336]
  [0.17600001 0.31999999 0.71666664]]

 [[0.46983334 0.31       0.65833336]
  [0.47933334 0.25       0.65833336]
  [0.46649998 0.41999999 0.64583331]
  ...
  [0.43450001 0.34       0.67916667]
  [0.44850001 0.31999999 0.67916667]
  [0.46783334 0.31       0.67916667]]

 ...

 [[0.46033335 0.63999999 0.66250002]
  [0.42633334 0.69       0.66250002]
  [0.39300001 0.73000002 0.66250002]
  ...
  [0.48083335 0.56999999 0.67083335]
  [0.46716669 0.56999999

In [ ]:
for i in range(10):
  num = np.random.randint(0, len(X_train))
  print("Sample", num)
  print(X_train[num])
  print("Actual temperature: ", y_temp_train[num])
  print("Actual humidity: ", y_humi_train[num])
  print("------")

Sample 39284
[[0.49233335 0.23       0.67916667]
 [0.48133332 0.56999999 0.67916667]
 [0.47966668 0.41999999 0.67916667]
 [0.46316668 0.56999999 0.67500001]
 [0.44933331 0.61000001 0.67916667]
 [0.42049998 0.69       0.67500001]
 [0.39333335 0.73000002 0.67916667]
 [0.37983334 0.69       0.67916667]
 [0.37150002 0.73000002 0.68333334]
 [0.35516664 0.73000002 0.68333334]
 [0.34483334 0.73000002 0.68333334]
 [0.34366667 0.73000002 0.67916667]
 [0.34033334 0.73000002 0.67916667]
 [0.33083335 0.93000001 0.67916667]
 [0.3256667  1.         0.67916667]
 [0.31933334 0.93000001 0.67916667]
 [0.31266668 0.93000001 0.67916667]
 [0.31199998 0.76999998 0.67916667]
 [0.34450001 0.73000002 0.67916667]
 [0.3725     0.69       0.67916667]
 [0.41300002 0.56999999 0.67916667]
 [0.45016667 0.50999999 0.67916667]
 [0.47566667 0.50999999 0.67916667]
 [0.49450001 0.47999999 0.67500001]]
Actual temperature:  0.49233335
Actual humidity:  0.23
------
Sample 11691
[[0.25466666 0.87       0.67500001]
 [0.2531666

In [ ]:
# train params
learning_rate = 1e-3
epochs = 200
batch_size = 128

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dropout, Dense, Conv1D, AveragePooling1D, Input, Flatten, SeparableConv1D

keras.backend.clear_session()

input = Input(shape=(window_size, 3, ))
x = Conv1D(8, 3, activation='relu', padding='same')(input)
x = AveragePooling1D(2)(x)
x = Conv1D(16, 3, activation='relu', padding='same')(x)
x = AveragePooling1D(2)(x)
x = Conv1D(32, 3, activation='relu', padding='same')(x)
x = AveragePooling1D(2)(x)
x = Conv1D(32, 3, activation='relu', padding='same')(x)
x = Conv1D(64, 3, activation='relu', padding='same')(x)
#x = GlobalAveragePooling1D()(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
temp_out = Dense(1, activation="linear", name="temperature")(x)
humi_out = Dense(1, activation="relu", name="humidity")(x)

model = keras.Model(inputs=input, outputs=[temp_out, humi_out], name="weather_model")

model.summary()

optim = tf.optimizers.AdamW(learning_rate = learning_rate)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_temperature_mean_squared_error', mode='min', factor=0.1, patience=20, min_lr=1e-6, verbose=1)
checkpoint_filepath = os.path.join(root_path, 'model_Conv1D.h5')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_temperature_mean_squared_error',
    mode='min',
    save_best_only=True,
    verbose=1)

model.compile(loss = {'temperature': 'mean_squared_error', 'humidity': 'mean_squared_error'},
              optimizer = optim, metrics = {'temperature': 'mean_squared_error', 'humidity': None})

model.fit(X_train, [y_temp_train, y_humi_train], validation_data=(X_test, [y_temp_test, y_humi_test]),
          epochs = epochs, batch_size = batch_size, shuffle=True, callbacks=[reduce_lr, model_checkpoint])

Model: "weather_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 3)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 24, 8)        80          ['input_1[0][0]']                
                                                                                                  
 average_pooling1d (AveragePool  (None, 12, 8)       0           ['conv1d[0][0]']                 
 ing1D)                                                                                           
                                                                                                  
 conv1d_1 (Conv1D)              (None, 12, 16)       400         ['average_pooling1d[0

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dropout, Dense, Conv1D, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, Reshape, Flatten

keras.backend.clear_session()

input = Input(shape=(window_size,3, ))
x = SeparableConv1D(8, 3, activation='relu', padding='same')(input)
x = AveragePooling1D(2)(x)
x = SeparableConv1D(16, 3, activation='relu', padding='same')(x)
x = AveragePooling1D(2)(x)
x = SeparableConv1D(32, 3, activation='relu', padding='same')(x)
x = AveragePooling1D(2)(x)
x = SeparableConv1D(32, 3, activation='relu', padding='same')(x)
x = SeparableConv1D(64, 3, activation='relu', padding='same')(x)
#x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
temp_out = Dense(1, activation="linear", name="temperature")(x)
humi_out = Dense(1, activation="relu", name="humidity")(x)

model = keras.Model(inputs=input, outputs=[temp_out, humi_out], name="weather_model")

model.summary()

optim = tf.optimizers.AdamW(learning_rate = learning_rate)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_temperature_mean_squared_error', mode='min', factor=0.1, patience=20, min_lr=1e-6, verbose=1)
checkpoint_filepath = os.path.join(root_path, 'model_SeparableConv1D.h5')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_temperature_mean_squared_error',
    mode='min',
    save_best_only=True,
    verbose=1)

model.compile(loss = {'temperature': 'mean_squared_error', 'humidity': 'mean_squared_error'},
              optimizer = optim, metrics = {'temperature': 'mean_squared_error', 'humidity': None})

model.fit(X_train, [y_temp_train, y_humi_train], validation_data=(X_test, [y_temp_test, y_humi_test]),
          epochs = epochs, batch_size = batch_size, shuffle=True, callbacks=[reduce_lr, model_checkpoint])

Model: "weather_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 3)]      0           []                               
                                                                                                  
 separable_conv1d (SeparableCon  (None, 24, 8)       41          ['input_1[0][0]']                
 v1D)                                                                                             
                                                                                                  
 average_pooling1d (AveragePool  (None, 12, 8)       0           ['separable_conv1d[0][0]']       
 ing1D)                                                                                           
                                                                                      

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dropout, Dense, Conv1D, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, Reshape, Flatten

keras.backend.clear_session()

input = Input(shape=(window_size, 3, ))
x = SeparableConv1D(8, 3, activation='relu', padding='same', dilation_rate=2)(input)
x = SeparableConv1D(16, 2, activation='relu', padding='same', dilation_rate=2)(x)
x = SeparableConv1D(32, 2, activation='relu', padding='same', dilation_rate=2)(x)
x = SeparableConv1D(32, 3, activation='relu', padding='same')(x)
x = SeparableConv1D(64, 3, activation='relu', padding='same')(x)
#x = GlobalAveragePooling1D()(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
temp_out = Dense(1, activation="linear", name="temperature")(x)
humi_out = Dense(1, activation="relu", name="humidity")(x)

model = keras.Model(inputs=input, outputs=[temp_out, humi_out], name="weather_model")

model.summary()

optim = tf.optimizers.AdamW(learning_rate = learning_rate)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_temperature_mean_squared_error', mode='min', factor=0.1, patience=20, min_lr=1e-6, verbose=1)
checkpoint_filepath = os.path.join(root_path, 'model_SeparableDilatedConv1D.h5')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_temperature_mean_squared_error',
    mode='min',
    save_best_only=True,
    verbose=1)

model.compile(loss = {'temperature': 'mean_squared_error', 'humidity': 'mean_squared_error'},
              optimizer = optim, metrics = {'temperature': 'mean_squared_error', 'humidity': None})

model.fit(X_train, [y_temp_train, y_humi_train], validation_data=(X_test, [y_temp_test, y_humi_test]),
          epochs = epochs, batch_size = batch_size, shuffle=True, callbacks=[reduce_lr, model_checkpoint])

Model: "weather_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 3)]      0           []                               
                                                                                                  
 separable_conv1d (SeparableCon  (None, 24, 8)       41          ['input_1[0][0]']                
 v1D)                                                                                             
                                                                                                  
 separable_conv1d_1 (SeparableC  (None, 24, 16)      160         ['separable_conv1d[0][0]']       
 onv1D)                                                                                           
                                                                                      

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dropout, Dense, Conv1D, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, Reshape, Flatten

keras.backend.clear_session()

input = Input(shape=(window_size,3, ))
x = Reshape(target_shape=(window_size*3,))(input)
x = Dense(128, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(32, activation="relu")(x)
temp_out = Dense(1, activation="linear", name="temperature")(x)
humi_out = Dense(1, activation="relu", name="humidity")(x)

model = keras.Model(inputs=input, outputs=[temp_out, humi_out], name="weather_model")

model.summary()

optim = tf.optimizers.AdamW(learning_rate = learning_rate)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_temperature_mean_squared_error', mode='min', factor=0.1, patience=30, min_lr=1e-6, verbose=1)
checkpoint_filepath = os.path.join(root_path, 'model_FC.h5')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_temperature_mean_squared_error',
    mode='min',
    save_best_only=True,
    verbose=1)

model.compile(loss = {'temperature': 'mean_squared_error', 'humidity': 'mean_squared_error'},
              optimizer = optim, metrics = {'temperature': 'mean_squared_error', 'humidity': None})

model.fit(X_train, [y_temp_train, y_humi_train], validation_data=(X_test, [y_temp_test, y_humi_test]),
          epochs = epochs, batch_size = batch_size, shuffle=True, callbacks=[reduce_lr, model_checkpoint])

Model: "weather_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 3)]      0           []                               
                                                                                                  
 reshape (Reshape)              (None, 72)           0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 128)          9344        ['reshape[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 128)          0           ['dense[0][0]']                  
                                                                                      

In [ ]:
import tensorflow as tf

def representative_dataset():
    for i in range(100):
      data = np.expand_dims(X_test[i], axis=0)
      yield [data.astype(np.float32)]

model_filename = 'model_FC.h5'
checkpoint_filepath = os.path.join(root_path, model_filename)
model = keras.models.load_model(checkpoint_filepath)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_type = tf.int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
tflite_filename = checkpoint_filepath.split('.')[0] + '.tflite'
with open(tflite_filename, 'wb') as f:
  f.write(tflite_quant_model)

In [ ]:
tfmicro_filename = checkpoint_filepath.split('.')[0] + '.h' # weather
!apt-get install -y xxd
!xxd -i $tflite_filename > $tfmicro_filename

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xxd is already the newest version (2:8.1.2269-1ubuntu5.15).
xxd set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
